In [0]:
# baseline model with weight decay on the cifar10 dataset
# CNN+Entropy Minimization+Mixup model on the cifar10 dataset
# Codes from below mentioned web site is used as a reference to build this model.
#https://machinelearningmastery.com/how-to-develop-a-cnn-from-scratch-for-cifar-10-photo-classification/
#We have also refered to the code from below github link
#https://github.com/yu4u/mixup-generator
# This link contains the mixup generator code we used 
#to augment data using mixup_generator and we have used it for our analysis
#Below is the licence copyright for mixup generator imported in this code
#MIT License
#Copyright (c) 2017 Yusuke Uchida

#@author : Jayesh Kudase, Prashant Jadhav, Aditya Chavan

import sys
from matplotlib import pyplot
from keras.datasets import cifar10
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.regularizers import l2
from mixup_generator import MixupGenerator
from keras.preprocessing.image import ImageDataGenerator
import keras.backend as k
import numpy as np
import sklearn.metrics as metrics
from sklearn.metrics import confusion_matrix
import seaborn as sns

#labeled_size=25000
#unlabeled_size=25000
labeled_size=15000
unlabeled_size=35000
# labeled_data_size=5000
# unlabeled_data_size=45000

# load train and test dataset
def load_dataset():
	# load dataset
	(train_X, train_Y), (test_X, test_Y1) = cifar10.load_data()
	# one hot encode target values
	train_Y = to_categorical(train_Y)
	test_Y = to_categorical(test_Y1)
	return train_X, train_Y, test_X, test_Y, test_Y1

# scale pixels
def prep_pixels(train, test):
	# convert from integers to floats
	training_norm = train.astype('float32')
	testing_norm = test.astype('float32')
	# normalize to range 0-1
	training_norm = training_norm / 255.0
	testing_norm = testing_norm / 255.0
	# return normalized images
	return training_norm, testing_norm


# define custom loss
def custom_loss(yTrue,yPred):
	print(yTrue[:1])
	lab_loss=k.sum(yTrue[:labeled_size]*k.log(yPred[:labeled_size]))/(-labeled_size)
	unlab_loss=k.sum(yPred[labeled_size:]*k.log(yPred[labeled_size:]))/(-unlabeled_size)
	return lab_loss+unlab_loss

# define cnn modez
def define_model():
	model = Sequential()
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=l2(0.001), input_shape=(32, 32, 3)))
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=l2(0.001)))
	model.add(MaxPooling2D((2, 2)))
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=l2(0.001)))
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=l2(0.001)))
	model.add(MaxPooling2D((2, 2)))
	model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=l2(0.001)))
	model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=l2(0.001)))
	model.add(MaxPooling2D((2, 2)))
	model.add(Flatten())
	model.add(Dense(128, activation='relu', kernel_initializer='he_uniform', kernel_regularizer=l2(0.001)))
	model.add(Dense(10, activation='softmax'))
	# compile model
	opt = SGD(lr=0.001, momentum=0.9)
	model.compile(optimizer=opt, loss=custom_loss, metrics=['accuracy'])
	
	return model

# plot diagnostic learning curves
def summarize_diagnostics(history):
	# plot loss
	pyplot.subplot(211)
	pyplot.title('Cross Entropy Loss')
	pyplot.plot(history.history['loss'], color='blue', label='train')
	pyplot.plot(history.history['val_loss'], color='orange', label='test')
	# plot accuracy
	pyplot.subplot(212)
	pyplot.title('Classification Accuracy')
	pyplot.plot(history.history['acc'], color='blue', label='train')
	pyplot.plot(history.history['val_acc'], color='orange', label='test')
	# save plot to file
	filename = sys.argv[0].split('/')[-1]
	pyplot.savefig(filename + '_plot.png')
	pyplot.close()

# run the test harness for evaluating a model
def run_test_harness():
	# load dataset
	train_X, train_Y, test_X, test_Y, test_Y1 = load_dataset()
	# prepare pixel data
	train_X, test_X = prep_pixels(train_X, test_X)
	# define model
	model = define_model()
	# fit model
	#history = model.fit(train_X, train_Y, epochs=100, batch_size=500, validation_data=(test_X, test_Y), verbose=1)
	datagen = ImageDataGenerator(
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True)
	training_generator = MixupGenerator(train_X, train_Y, batch_size=500, alpha=0.2, datagen=datagen)()
	history=model.fit_generator(generator=training_generator,
                        steps_per_epoch=500 ,
                        validation_data=(test_X, test_Y),
                        epochs=50, verbose=1)
	#confusion matrix
	y_pred_ohe = model.predict(test_X) 
	y_pred_labels = np.argmax(y_pred_ohe, axis=1) 
	confusion_matrix = metrics.confusion_matrix(y_true=test_Y1, y_pred=y_pred_labels)
	figure = pyplot.figure(figsize=(8, 8))
	sns.heatmap(confusion_matrix, annot=False,cmap=pyplot.cm.Blues)
	pyplot.tight_layout()
	pyplot.ylabel('True label')
	pyplot.xlabel('Predicted label')
	pyplot.imshow(confusion_matrix)
	# evaluate model
	_, acc = model.evaluate(test_X, test_Y, verbose=0)
	print('> %.3f' % (acc * 100.0))
	# learning curves
	#summarize_diagnostics(history)

# entry point, run the test harness
run_test_harness()

Using TensorFlow backend.


170500096/170498071 [==============================] - 11s 0us/step





Tensor("loss/dense_2_loss/strided_slice:0", shape=(?, ?), dtype=float32)




Epoch 1/50





500/500 [==============================] - 157s 314ms/step - loss: 1.2157 - acc: 0.1844 - val_loss: 1.1997 - val_acc: 0.2511
Epoch 2/50
500/500 [==============================] - 150s 300ms/step - loss: 1.1883 - acc: 0.2558 - val_loss: 1.1743 - val_acc: 0.2970
Epoch 3/50
500/500 [==============================] - 150s 300ms/step - loss: 1.1641 - acc: 0.2853 - val_loss: 1.1502 - val_acc: 0.3178
Epoch 4/50
500/500 [==============================] - 152s 304ms/step - loss: 1.1411 - acc: 0.3036 - val_loss: 1.1272 - val_acc: 0.3393
Epoch 5/50
500/500 [==============================] - 152s 304ms/step - loss: 1.1190 - acc: 0.3167 - val_loss: 1.1052 - val_acc: 0.3531
Epoch 6/50
500/500 [==============================] - 152s 305ms/step - loss: 1.0975 - acc: 0.3266 - val_loss: 1.0837 - val_acc: 0.3629
Epoch 7/50
500/500 [=========

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-1-bb4ccd2df8ef>", line 129, in <module>
    run_test_harness()
  File "<ipython-input-1-bb4ccd2df8ef>", line 111, in run_test_harness
    epochs=50, verbose=1)
  File "/usr/local/lib/python3.6/dist-packages/keras/legacy/interfaces.py", line 91, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/keras/engine/training.py", line 1658, in fit_generator
    initial_epoch=initial_epoch)
  File "/usr/local/lib/python3.6/dist-packages/keras/engine/training_generator.py", line 181, in fit_generator
    generator_output = next(output_generator)
  File "/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py", line 702, in get
    inputs = future.get(timeout=30)
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 638, in get
    

KeyboardInterrupt: ignored